In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

from tqdm import tqdm
import torch.optim as optim

from utils import train, plot_images_full, generate_matrix, batch_dirichlet, get_similarities, plot_similarities, get_differences, plot_differences, get_images, plot_images

#from models.vit import ViT, SimpleViT, SimpleViTI, SimpleViTFT
from models.diffusion import SimpleTransformer

from sklearn.metrics.pairwise import cosine_similarity

/home/gbetondji/anaconda3/envs/smoothtransformer/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH = "model_one.pth"
batch_size = 32
size = (3,32,32)
patch_size = 1
depth = 128
device = 'cpu'
data_path = './data/'
num_epochs = 100
learning_rate = 1e-3

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root=data_path, train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=data_path, train=False,
                                       download=True, transform=transform)
val_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


## 1

In [4]:
model = SimpleTransformer(size, depth, 10, sign=1, tau=1, weight_sharing=True, method='A',stdev=0.01)

In [5]:
model.to(device)

SimpleTransformer(
  (WVs): ParameterList(  (0): Parameter containing: [torch.float32 of size 3x3])
  (WKs): ParameterList(  (0): Parameter containing: [torch.float32 of size 3x3])
  (WQs): ParameterList(  (0): Parameter containing: [torch.float32 of size 3x3])
  (last_layer): Linear(in_features=1024, out_features=10, bias=True)
)

In [10]:
next(model.parameters()).is_cuda

True

In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

train_accs_list = []
train_loss_list = []
val_accs_list = []
val_loss_list = []

for epoch in tqdm(range(1, num_epochs + 1), desc="Epoch"):  
    train_loss, train_acc = train(model,train_loader,optimizer,criterion,epoch,device)  

    # Model validation
    model.eval()
    val_loss = val_acc = 0.0
    for (img, labels) in val_loader:
        img, labels = img.to(device), labels.to(device)
        with torch.no_grad():
            predictions = model(img)
            loss = criterion(predictions, labels)
            correct = torch.argmax(predictions.data, 1) == labels
        val_loss += loss
        val_acc += correct.sum()
    val_loss /= len(val_loader.dataset)
    val_acc /= len(val_loader.dataset)
    
    train_accs_list.append(train_acc)
    val_accs_list.append(val_acc)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)

    print()
    print(f" Train acc {train_acc:.3f}, Train loss {train_loss:.6f}")
    print(f" Valid acc {val_acc:.3f}, Val loss {val_loss:.6f}")
    
print(f"Trained {config.method}")

Epoch:   0%|                                                                                                                                            | 0/100 [13:59<?, ?it/s]


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [9]:
torch.save(model.state_dict(),PATH)

In [12]:
model.load_state_dict(torch.load(PATH))

AttributeError: '_IncompatibleKeys' object has no attribute 'load_state_dict'